<a href="https://colab.research.google.com/github/sam80FDS/Sam/blob/main/Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#  HEADLESS STUDIO v29.0 (The Omni-Patch)
#  Engine: ComfyUI + GitMyLo + ParlerTTS
#  Fixes: Missing 'funcy', 'encodec', 'audiotools', and 'torch' conflicts.
# ==============================================================================

import os
import time

print("------------------------------------------------")
print("STEP 1: SYSTEM LAYER")
print("------------------------------------------------")
!apt-get update
!apt-get install -y ffmpeg libsndfile1 libavcodec-dev libavformat-dev libavutil-dev libswscale-dev libavdevice-dev pkg-config

print("------------------------------------------------")
print("STEP 2: APPLICATION LAYER")
print("------------------------------------------------")
%cd /content
!rm -rf ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -r requirements.txt

print("------------------------------------------------")
print("STEP 3: NODE INJECTION")
print("------------------------------------------------")
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git
!git clone https://github.com/gitmylo/ComfyUI-audio-nodes.git
!git clone https://github.com/smthemex/ComfyUI_ParlerTTS.git

print("------------------------------------------------")
print("STEP 4: FORCED DEPENDENCY INSTALLATION (The Fix)")
print("------------------------------------------------")
# 1. FIX AUDIO-NODES (GitMyLo)
%cd /content/ComfyUI/custom_nodes/ComfyUI-audio-nodes

# NUCLEAR STRIKE: Delete the bad manifest
!rm requirements.txt

# MANUAL INSTALL: We explicitly install 'funcy', 'encodec', and others.
# This fixes the error in your screenshot.
!pip install funcy encodec soundfile numpy scipy librosa

# FORCE AUDIOCRAFT (No Dependencies Mode)
!pip install --no-deps git+https://github.com/facebookresearch/audiocraft.git

# 2. FIX PARLER TTS
%cd ../ComfyUI_ParlerTTS
!pip install -r requirements.txt
# Manually install audiotools to prevent the other common error
!pip install git+https://github.com/descriptinc/audiotools.git

print("------------------------------------------------")
print("STEP 5: ASSET DOWNLOAD")
print("------------------------------------------------")
%cd /content/ComfyUI/models/checkpoints
!wget -nc -O dreamshaper_8.safetensors https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors
!wget -nc -O svd_xt.safetensors https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors
!wget -nc -O musicgen_small.safetensors https://huggingface.co/facebook/musicgen-small/resolve/main/model.safetensors

!mkdir -p /content/ComfyUI/models/parler_tts
!wget -nc -O /content/ComfyUI/models/parler_tts/parler-tts-mini-v1.safetensors https://huggingface.co/parler-tts/parler-tts-mini-v1/resolve/main/model.safetensors

!mkdir -p /content/ComfyUI/models/upscale_models
!wget -nc -O /content/ComfyUI/models/upscale_models/RealESRGAN_x4plus.pth https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
!mkdir -p /content/ComfyUI/models/vfi
!wget -nc -O /content/ComfyUI/models/vfi/rife47.pth https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife47.pth

print("------------------------------------------------")
print("STEP 6: LAUNCH")
print("------------------------------------------------")
%cd /content/ComfyUI
!wget -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

print("ENGINE INITIALIZING... WAIT FOR THE LINK")
!python main.py --listen --dont-print-server & cloudflared tunnel --url http://localhost:8188
